In the modular approach I will create separate files for different model types. This one will be for Beta Sil App Score

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench_dimfact

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """
    Calculate Gini coefficient with proper handling of edge cases
    
    Returns np.nan when:
    - Fewer than 2 observations
    - No positive labels (sum of labels = 0)
    """
    n = len(scores)
    if n < 2:
        return np.nan
    
    label_sum = np.sum(labels)
    
    # Handle case where no positive labels exist (all zeros)
    # This prevents division by zero warning
    if label_sum == 0:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * label_sum)
    return gini


def calculate_periodic_gini_prod_ver_trench_dimfact(df, score_column, label_column, namecolumn, 
                                        data_selection_column=None,
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        ostype_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients and return Power BI-friendly long format
    with fact and dimension tables.
    
    Returns:
    - fact_table: Long format with one row per segment per period
    - dimension_table: Unique segment combinations for filtering
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    data_selection_column: (optional) name of column for data selection (Test/Train)
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    ostype_column: (optional) name of column for OS type
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'data_selection': data_selection_column,
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'ostype': ostype_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if data_selection_column:
        segment_columns.append(('DataSelection', data_selection_column))
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    if ostype_column:
        segment_columns.append(('OSType', ostype_column))
    
    # Generate all possible combinations of segment columns
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini_value')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        else:
            weekly_gini['distinct_accounts'] = None
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini_value')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        else:
            monthly_gini['distinct_accounts'] = None
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        gini_results['data_selection'] = metadata.get(data_selection_column, None) if data_selection_column else None
        gini_results['model_version'] = metadata.get(model_version_column, None) if model_version_column else None
        gini_results['trench_category'] = metadata.get(trench_column, None) if trench_column else None
        gini_results['loan_type'] = metadata.get(loan_type_column, None) if loan_type_column else None
        gini_results['loan_product_type'] = metadata.get(loan_product_type_column, None) if loan_product_type_column else None
        gini_results['ostype'] = metadata.get(ostype_column, None) if ostype_column else None
        
        all_results.append(gini_results)
    
    # Combine all results
    fact_table = pd.concat(all_results, ignore_index=True)
    
    # Create dimension table (unique segment combinations for filtering)
    dimension_table = fact_table[['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                   'trench_category', 'loan_type', 'loan_product_type', 'ostype']].drop_duplicates().reset_index(drop=True)
    dimension_table['segment_id'] = range(len(dimension_table))
    
    # Add segment_id to fact table
    fact_table = fact_table.merge(dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 
                                                     'data_selection', 'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']], 
                                  on=['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                      'trench_category', 'loan_type', 'loan_product_type', 'ostype'], 
                                  how='left')
    
    # Reorder columns in fact table
    fact_table = fact_table[['segment_id', 'start_date', 'end_date', 'period', 'gini_value', 'distinct_accounts',
                             'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 'trench_category', 
                             'loan_type', 'loan_product_type', 'ostype']]
    
    # Reorder columns in dimension table
    dimension_table = dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 
                                        'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']]
    
    return fact_table, dimension_table


# Usage:
# fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     data_selection_column='Data_selection',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     ostype_column='osType',
#     account_id_column='digitalLoanAccountId'
# )
# 
# # In Power BI:
# # 1. Import fact_table and dimension_table
# # 2. Create relationship: dimension_table[segment_id] -> fact_table[segment_id]
# # 3. Use dimension table columns as filters (including ostype)
# # 4. Create DAX measures:
# #    - Gini Measure = AVERAGE(fact_table[gini_value])
# #    - Account Count = SUM(fact_table[distinct_accounts])
# # 5. Use start_date, end_date, period for time-based analysis

# update_tables

In [4]:

def update_tables(fact_table: pd.DataFrame, dimension_table: pd.DataFrame, model_name: str, product: str) -> tuple:
    """
    Updates fact_table and dimension_table:
    - Sets 'Model_display_name' to the given model_name
    - Replaces NaN values in specified columns with 'Overall'
    
    Returns:
        Updated fact_table and dimension_table as a tuple
    """
    # Columns where missing values should be replaced
    cols_to_replace = ['model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']
    
    # Update fact_table
    fact_table['Model_display_name'] = model_name
    fact_table['Product_Category'] = product
    fact_table[cols_to_replace] = fact_table[cols_to_replace].fillna('Overall')
    
    # Update dimension_table
    dimension_table['Model_display_name'] = model_name
    dimension_table['Product_Category'] = product
    dimension_table[cols_to_replace] = dimension_table[cols_to_replace].fillna('Overall')
    
    return fact_table, dimension_table


## Beta SIL Demo Score 

### FPD0 

#### Test 

In [5]:
sq = """ 
WITH cleaned AS (
    SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
deviceOs osType,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
  osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and flg_mature_fpd0 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (124524, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3802932,014f2b59-b335-4e68-8388-a3e7210816fc,60838029320014,beta_demo_model_sil,0.0640769224,2025-11-10 15:45:42,2025-11-10,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,3714357,01e37ef8-14bb-4f3b-9d5c-a329029686f1,60837143570012,beta_demo_model_sil,0.06691913,2025-09-30 11:54:11,2025-09-30,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3556200,023b10ac-e652-4e64-9c15-11cb9a3f0585,60835562000018,beta_demo_model_sil,0.0607173587,2025-07-14 11:38:53,2025-07-14,2025-07,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
3,3729272,03b830bd-e117-4aae-9f95-96ddc36df7d3,60837292720013,beta_demo_model_sil,0.0691205642,2025-10-07 13:44:37,2025-10-07,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
4,3714410,03fedbdd-941a-46e5-a9ec-153299050f0f,60837144100011,beta_demo_model_sil,0.0516157641,2025-09-30 12:18:28,2025-09-30,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android


In [6]:
df1 = dfd.copy()

### Train 

In [7]:
sq = """ 
WITH cleaned AS (
    SELECT
  mmrd.customerId, mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
Data_selection,
   deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
    left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory,Data_selection, 
     osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
         coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and flg_mature_fpd0 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (432868, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,1875530,01f87d12-d85e-4c2a-bc87-872f8d8f3388,60818755300013,beta_demo_model_sil,0.121127,2023-01-27 10:16:57,2023-01-27,2023-01,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,2208264,0274d008-00d5-4486-acfb-ce536a3a9e8e,60822082640012,beta_demo_model_sil,0.171407,2023-08-26 15:11:40,2023-08-26,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,ios
2,2185194,06b5f9e9-892a-4b3d-a7d6-3e2535b962a1,60821851940017,beta_demo_model_sil,0.073951,2023-08-12 11:04:17,2023-08-12,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,2185659,08ab4a7f-da76-4e48-857d-78681af18286,60821856590019,beta_demo_model_sil,0.084582,2023-08-12 15:41:07,2023-08-12,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2177412,0f5f8f9e-f2c5-4e88-9c09-3b4a48103a6f,60821774120011,beta_demo_model_sil,0.110040,2023-08-07 14:02:26,2023-08-07,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [8]:
df2 = dfd.copy()

#### Concatenate Test and Train Datasets 

In [9]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (494196, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3802932,014f2b59-b335-4e68-8388-a3e7210816fc,60838029320014,beta_demo_model_sil,0.0640769224,2025-11-10 15:45:42,2025-11-10,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,2597193,04d2fbdb-3e10-4aff-8499-7da0ebf02ea8,60825971930039,beta_demo_model_sil,0.0774549021,2025-11-06 14:22:06,2025-11-06,2025-11,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,3780257,0680d1ba-7ec8-4fad-94f1-996e5a8e24b7,60837802570018,beta_demo_model_sil,0.0442327321,2025-10-30 17:43:12,2025-10-30,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
3,3822608,08e9a201-9333-4300-be70-a360782b76dc,60838226080015,beta_demo_model_sil,0.0470225212,2025-11-20 14:38:20,2025-11-20,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,ios
4,3802360,0e528211-ebde-420b-8feb-1680b3433cbe,60838023600011,beta_demo_model_sil,0.0666446954,2025-11-10 12:46:18,2025-11-10,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


#### Making sure the score is numerical

In [10]:
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [11]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd0', 
    'FPD0',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

In [12]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_demo_model_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (80053, 17)
The shape of the dimension table is:	 (1873, 12)


In [13]:
facttable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
dimtable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"

In [14]:
# Upload to BigQuery

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=89b50a7f-7fa2-49ad-9f62-50f72d79187a>

In [15]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=78eedd35-d93e-4b51-84b5-86272983e358>

### FPD10 

### Test 

In [16]:
sq = """ 
WITH cleaned AS (
     SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
    deviceOs osType,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd10 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (112103, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3641154,000694f7-daef-4d10-b255-951570074bd8,60836411540011,beta_demo_model_sil,0.0215581599,2025-08-24 16:39:28,2025-08-24,2025-08,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
1,2422071,0007f464-8632-4319-99c4-262a92e57540,60824220710021,beta_demo_model_sil,0.079982085,2025-09-05 17:12:09,2025-09-05,2025-09,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android
2,3501806,007b9e45-0cae-41ae-b204-587619fedd35,60835018060017,beta_demo_model_sil,0.0449859679,2025-06-16 18:32:28,2025-06-16,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,3748100,00e9c622-fb99-41b2-a8fb-89bd03709add,60837481000011,beta_demo_model_sil,0.0512348094,2025-10-16 14:38:20,2025-10-16,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
4,3482698,01385104-20fc-4f15-8b4e-a3c5ba7fc0ba,60834826980019,beta_demo_model_sil,0.0252304527,2025-06-07 10:34:59,2025-06-07,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [17]:
df1 = dfd.copy()

### Train 

In [18]:
sq = """ 
WITH cleaned AS (
    SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
    left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory,Data_selection,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd10 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (432868, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2001690,022fd594-0604-4614-9d95-a4e53a5b8fc5,60820016900016,beta_demo_model_sil,0.112647,2023-04-18 14:30:01,2023-04-18,2023-04,Pre_Train,1,1,SIL-Instore,v1,Trench 2,Appliance,android
1,1878307,035290db-826a-4021-a908-636a5fa50ed9,60818783070011,beta_demo_model_sil,0.041235,2023-01-29 17:50:07,2023-01-29,2023-01,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,2176213,0433813b-faed-4043-8618-f49bad9f9d63,60821762130012,beta_demo_model_sil,0.077685,2023-08-06 18:13:27,2023-08-06,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
3,2185298,05affd6e-5689-4c80-bdd0-559ec25b9569,60821852980014,beta_demo_model_sil,0.098407,2023-08-12 11:57:30,2023-08-12,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2100090,0aa5e413-d15a-48fa-ad9c-e6ac5ef1cbfa,60821000900011,beta_demo_model_sil,0.068587,2023-06-19 18:58:19,2023-06-19,2023-06,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [19]:
df2 = dfd.copy()

#### Concatenate Test and Train Set

In [20]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (481775, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3748100,00e9c622-fb99-41b2-a8fb-89bd03709add,60837481000011,beta_demo_model_sil,0.0512348094,2025-10-16 14:38:20,2025-10-16,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,3808709,0582c91a-45d0-445f-a4ef-97935668ff35,60838087090013,beta_demo_model_sil,0.123970089,2025-11-13 17:45:12,2025-11-13,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,2764335,07fafe16-51d5-438f-88b4-8412c4e84acb,60827643350022,beta_demo_model_sil,0.1226818984,2025-11-18 18:24:22,2025-11-18,2025-11,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android
3,3509855,0bd932d2-0ff5-45a7-8a56-c0665902da82,60835098550018,beta_demo_model_sil,0.0657062068,2025-10-28 19:10:22,2025-10-28,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Mall,android
4,3757229,0c35b997-a2fd-4a17-9f9c-372c1f603a67,60837572290017,beta_demo_model_sil,0.0877065553,2025-10-20 17:53:44,2025-10-20,2025-10,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


#### Making sure the score is numerical

In [21]:
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

#### Calculate Gini

In [22]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd10', 
    'FPD10',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
      ostype_column='osType',    
    account_id_column='digitalLoanAccountId'
)

#### Update fact and dimension table

In [23]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_demo_model_sil', product='SIL')

In [24]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b87926ef-8a8f-4345-8485-9645a3675953>

In [25]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c10d3deb-bd1f-42df-80c8-9f26833a3065>

### FPD30 

### Test 

In [26]:
sq = """ 
WITH cleaned AS (
 SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
   deviceOs osType,
   
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
     osType,
    from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        
    coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (93233, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2422071,0007f464-8632-4319-99c4-262a92e57540,60824220710021,beta_demo_model_sil,0.079982085,2025-09-05 17:12:09,2025-09-05,2025-09,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android
1,3501806,007b9e45-0cae-41ae-b204-587619fedd35,60835018060017,beta_demo_model_sil,0.0449859679,2025-06-16 18:32:28,2025-06-16,2025-06,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,3748100,00e9c622-fb99-41b2-a8fb-89bd03709add,60837481000011,beta_demo_model_sil,0.0512348094,2025-10-16 14:38:20,2025-10-16,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
3,3482698,01385104-20fc-4f15-8b4e-a3c5ba7fc0ba,60834826980019,beta_demo_model_sil,0.0252304527,2025-06-07 10:34:59,2025-06-07,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3503217,01a22baa-098b-45ed-9513-ac58af8acdbd,60835032170011,beta_demo_model_sil,0.1035191896,2025-06-17 14:33:40,2025-06-17,2025-06,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android


In [27]:
df1 = dfd.copy()

### Train 

In [28]:
sq = """ 
WITH cleaned AS (
    SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
   case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
    left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory, Data_selection,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fpd30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (432868, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2064034,004286cb-e3e3-4203-99e0-c41581ef29a3,60820640340012,beta_demo_model_sil,0.135406,2023-05-27 16:45:48,2023-05-27,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,2187117,0222a885-4fcf-4f5a-b4cd-0cfabd0e6cca,60821871170012,beta_demo_model_sil,0.082244,2023-08-13 13:47:39,2023-08-13,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,2014813,073cd7ff-2c69-4089-b70f-3999a9ac9494,60820148130016,beta_demo_model_sil,0.133816,2023-04-26 15:14:32,2023-04-26,2023-04,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,2188617,08662442-cc93-4128-88a0-2896e10d2812,60821886170011,beta_demo_model_sil,0.166866,2023-08-14 12:17:20,2023-08-14,2023-08,Dev_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
4,2148353,0b48777a-f2cb-42ba-b04a-fc7609e31558,60821483530016,beta_demo_model_sil,0.101154,2023-07-21 11:33:56,2023-07-21,2023-07,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [29]:
df2 = dfd.copy()

### Concat 

In [30]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (462905, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3748100,00e9c622-fb99-41b2-a8fb-89bd03709add,60837481000011,beta_demo_model_sil,0.0512348094,2025-10-16 14:38:20,2025-10-16,2025-10,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,3763526,18b896ec-52bf-4b99-81f4-65dcb6a6de76,60837635260012,beta_demo_model_sil,0.063815527,2025-10-23 17:29:31,2025-10-23,2025-10,Prod,1,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3800895,190bafdc-d2c5-4075-94f7-e01d2f26ba88,60838008950018,beta_demo_model_sil,0.0434380631,2025-11-09 13:03:11,2025-11-09,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
3,3851786,1911b3cd-e37c-4ad4-9fdf-c2ef2315a1dd,60838517860013,beta_demo_model_sil,0.0403460169,2025-12-01 15:50:15,2025-12-01,2025-12,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3109796,21a904de-ed57-4a28-a9f2-06d81ddeb85e,60831097960028,beta_demo_model_sil,0.0878811516,2025-10-21 15:56:18,2025-10-21,2025-10,Prod,0,1,SIL Competitor,v1,Trench 3,Appliance,android


#### Making sure the score is numerical

In [31]:
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

#### Calculate the Gini

In [32]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'sil_beta_demo_score', 
    'deffpd30', 
    'FPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

#### Updating fact and dimension tables

In [33]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_demo_model_sil', product='SIL')

#### Inserting into the fact and dimension table

In [34]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=12f1b812-9e2f-4e42-ba2b-36f910578a61>

In [35]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0eb6f395-8e4f-4faa-a102-822bd3190f83>

### FSPD30 

### Test 

In [36]:
sq = """ 
WITH cleaned AS (
     SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
   deviceOs osType,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
     osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (71708, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3494376,0257c77c-7ecc-4246-b68f-d901a1ec66fd,60834943760017,beta_demo_model_sil,0.0372536042,2025-06-13 11:52:19,2025-06-13,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Mall,android
1,3676532,0971d6ae-4644-416a-a017-150320cab414,60836765320011,beta_demo_model_sil,0.0305769925,2025-09-10 17:58:31,2025-09-10,2025-09,Prod,0,1,SIL Competitor,v1,Trench 2,Appliance,android
2,3704161,0e98bea9-17ec-4b32-849c-7ad2cfd16715,60837041610017,beta_demo_model_sil,0.1349205076,2025-09-25 12:36:24,2025-09-25,2025-09,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3488457,12136f67-f893-4c64-ab49-6d925d2c5ca5,60834884570018,beta_demo_model_sil,0.0604840108,2025-06-10 09:09:57,2025-06-10,2025-06,Prod,0,1,SIL ZERO,v1,Trench 2,Appliance,android
4,3501411,1f66f97a-b704-4ad2-8f8d-429b891ba201,60835014110012,beta_demo_model_sil,0.0859889286,2025-06-16 16:06:24,2025-06-16,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [37]:
df1 = dfd.copy()

### Train 

In [38]:
sq = """ 
WITH cleaned AS (
    SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
    left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory, Data_selection,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (432868, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2078571,01cd7b8f-a5f9-4741-8c1c-c01a47997734,60820785710013,beta_demo_model_sil,0.115201,2023-06-05 12:18:25,2023-06-05,2023-06,Pre_Train,1,1,SIL-Instore,v1,Trench 2,Appliance,android
1,2020778,01d62bb7-efbc-4a87-9a3c-ff3c2f65dbde,60820207780019,beta_demo_model_sil,0.080503,2023-04-30 12:40:40,2023-04-30,2023-04,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,2086602,0689b962-26b9-43b0-930f-7e1b9ede029c,60820866020011,beta_demo_model_sil,0.102917,2023-06-10 16:27:21,2023-06-10,2023-06,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,1851150,0bc25504-db62-4e19-8571-785901958b82,60818511500019,beta_demo_model_sil,0.151433,2023-01-05 13:39:00,2023-01-05,2023-01,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,2116131,11e0850d-3d7e-4222-9129-932f05dbdc6f,60821161310011,beta_demo_model_sil,0.074568,2023-07-02 12:39:33,2023-07-02,2023-07,Dev_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [39]:
df2 = dfd.copy()

#### Concatenate Test and Train Dataset

In [40]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()

# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (441380, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3784717,3b2c6c33-1f4c-4abd-84bd-ab50523fc5e3,60837847170011,beta_demo_model_sil,0.0242049859,2025-11-01 16:23:44,2025-11-01,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Mall,android
1,3784240,7f116197-2c26-490a-8345-942fec853490,60837842400015,beta_demo_model_sil,0.0941062053,2025-11-01 13:45:43,2025-11-01,2025-11,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
2,3784729,90018901-36d9-4eef-8194-d4c61c986f6c,60837847290015,beta_demo_model_sil,0.0303731883,2025-11-01 16:26:35,2025-11-01,2025-11,Prod,0,1,SIL Competitor,v1,Trench 2,Mall,ios
3,3783930,9af3fa5e-7c00-4f0a-878d-8ce4a5c4d344,60837839300011,beta_demo_model_sil,0.0533529356,2025-11-01 12:10:02,2025-11-01,2025-11,Prod,1,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3784299,2e7258d9-ccaa-47da-b852-b82dffb68d88,60837842990014,beta_demo_model_sil,0.0954058158,2025-11-01 14:51:10,2025-11-01,2025-11,Prod,1,1,SIL Competitor,v1,Trench 2,Mall,android


In [41]:
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [42]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'sil_beta_demo_score', 
    'deffspd30', 
    'FSPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

In [43]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_demo_model_sil', product='SIL')

In [44]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=487dad4f-d9fc-4b21-882c-d70cd2c32daf>

In [45]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=35e6ee3b-6012-45ee-9174-218cf9dc7d75>

### FSTPD30 

### Test 

In [46]:
sq = """ 
WITH cleaned AS (
SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
    deviceOs osType,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details` mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  modelDisplayName,
  modelVersionId, trenchCategory,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select *  from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (56327, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3348744,02909311-ca08-45a8-a5ef-ecb99873668a,60833487440016,beta_demo_model_sil,0.0319824289,2025-03-30 12:19:04,2025-03-30,2025-03,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
1,2744135,02e4edb7-a5de-4415-a034-07b63370b61b,60827441350026,beta_demo_model_sil,0.0785887604,2025-05-01 18:50:22,2025-05-01,2025-05,Prod,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,3472051,0f123bcd-4a6e-41fd-8d06-6e0b902b7b57,60834720510011,beta_demo_model_sil,0.018935209,2025-06-01 16:01:23,2025-06-01,2025-06,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
3,3644242,102d7e95-06e7-4c50-b116-837022cb3178,60836442420011,beta_demo_model_sil,0.0458955167,2025-08-26 10:51:01,2025-08-26,2025-08,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,3391343,213ac559-d6c0-41c6-854c-80ecf9bb3aaf,60833913430011,beta_demo_model_sil,0.0655131675,2025-04-20 16:35:07,2025-04-20,2025-04,Prod,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [47]:
df1 = dfd.copy()

### Train 

In [48]:
sq = """ 
WITH cleaned AS (
  SELECT
  mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned,
Data_selection,
    deviceOs osType,
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
    left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ), 
  modelname as (
  select customerId,
  digitalLoanAccountId,
  start_time,
  prediction sil_beta_demo_score,
  case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName,
  modelVersionId, trenchCategory, Data_selection,
      osType,
  from cleaned
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.sil_beta_demo_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
    del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.sil_beta_demo_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select * from base
  qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
  ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (425999, 16)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2091420,008fad6a-52d1-4300-8f16-0fe4f0bdfe52,60820914200011,beta_demo_model_sil,0.107534,2023-06-13 17:58:02,2023-06-13,2023-06,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,2042763,09fe8a45-96ed-4471-a633-1d8e5bed6a8a,60820427630011,beta_demo_model_sil,0.064595,2023-05-13 17:51:24,2023-05-13,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,1886279,0b0d6519-c127-434b-8e99-73d6488ee663,60818862790011,beta_demo_model_sil,0.102550,2023-02-04 15:53:26,2023-02-04,2023-02,Pre_Train,1,1,SIL-Instore,v1,Trench 2,Appliance,ios
3,1918655,0ef5fcb1-23ab-4876-8775-5290c1f6db19,60819186550013,beta_demo_model_sil,0.131192,2023-02-26 18:32:38,2023-02-26,2023-02,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,1855075,15475e9a-18ce-4369-8b65-77ed6388a5ba,60818550750011,beta_demo_model_sil,0.228963,2023-01-08 18:23:49,2023-01-08,2023-01,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [49]:
df2 = dfd.copy()

### Concat 

In [50]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()

# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (425999, 16)


C:\Users\Dwaipayan\AppData\Local\Temp\ipykernel_30292\436855073.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,sil_beta_demo_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2091420,008fad6a-52d1-4300-8f16-0fe4f0bdfe52,60820914200011,beta_demo_model_sil,0.107534,2023-06-13 17:58:02,2023-06-13,2023-06,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,ios
1,2042763,09fe8a45-96ed-4471-a633-1d8e5bed6a8a,60820427630011,beta_demo_model_sil,0.064595,2023-05-13 17:51:24,2023-05-13,2023-05,Pre_Train,0,1,SIL-Instore,v1,Trench 3,Appliance,android
2,1886279,0b0d6519-c127-434b-8e99-73d6488ee663,60818862790011,beta_demo_model_sil,0.102550,2023-02-04 15:53:26,2023-02-04,2023-02,Pre_Train,1,1,SIL-Instore,v1,Trench 2,Appliance,ios
3,1918655,0ef5fcb1-23ab-4876-8775-5290c1f6db19,60819186550013,beta_demo_model_sil,0.131192,2023-02-26 18:32:38,2023-02-26,2023-02,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android
4,1855075,15475e9a-18ce-4369-8b65-77ed6388a5ba,60818550750011,beta_demo_model_sil,0.228963,2023-01-08 18:23:49,2023-01-08,2023-01,Pre_Train,0,1,SIL-Instore,v1,Trench 2,Appliance,android


In [51]:
df_concat['sil_beta_demo_score'] = pd.to_numeric(df_concat['sil_beta_demo_score'], errors='coerce')

In [52]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'sil_beta_demo_score', 
    'deffstpd30', 
    'FSTPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [53]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_demo_model_sil', product='SIL')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (70505, 17)
The shape of the dimension table is:	 (1510, 12)


In [54]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=29dbe65d-cd22-4371-b14d-3504536e3b29>

In [55]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=dd63fba0-b31e-4dde-bffa-1ec830b5fe6a>